# This program can plot u_e_ij for a $10 \times 10$ mesh.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

from matplotlib import pyplot as plt
import numpy as np
import arrayfire as af
import h5py

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh
from dg_maxwell import wave_equation_2d
from dg_maxwell import lagrange

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

[[ 0  9]
 [10 19]
 [20 29]
 [30 39]
 [40 49]
 [50 59]
 [60 69]
 [70 79]
 [80 89]
 [90 99]]
[[ 0 90]
 [ 1 91]
 [ 2 92]
 [ 3 93]
 [ 4 94]
 [ 5 95]
 [ 6 96]
 [ 7 97]
 [ 8 98]
 [ 9 99]]
gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [2]:
params.mesh_file = '../read_and_plot_mesh/mesh/square_10_10.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)
new_elements = msh_parser.rearrange_element_edges(advec_var.elements, advec_var)
advec_var.reassign_2d_elements(new_elements)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 441 nodes.
gmshTranslator: Mesh has 144 elements.
gmshTranslator: Processed 589 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


In [7]:
def contour_2d(u, advec_var, ax, title, fontsize = 12):
    '''
    '''
    color_levels = np.linspace(-3.0, 3.0, 200)
    u_plot = af.flip(af.moddims(u, params.N_LGL, params.N_LGL, 10, 10), 0)
    x_plot = af.flip(af.moddims(advec_var.x_e_ij, params.N_LGL, params.N_LGL, 10, 10), 0)
    y_plot = af.flip(af.moddims(advec_var.y_e_ij, params.N_LGL, params.N_LGL, 10, 10), 0)


    x_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    y_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    u_contour = af.np_to_af_array(np.zeros([params.N_LGL * 10, params.N_LGL * 10]))
    #
    for r in range(100):
        p = int(r / 10)
        q = r - p * 10
        x_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = x_plot[:, :, q, p]

        y_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = y_plot[:, :, q, p]

        u_contour[p * params.N_LGL:params.N_LGL * (p + 1),\
                  q * params.N_LGL:params.N_LGL * (q + 1)] = u_plot[:, :, q, p]

    x_contour = np.array(x_contour)
    y_contour = np.array(y_contour)
    u_contour = np.array(u_contour)
    im = ax.contourf(x_contour, y_contour, u_contour, 200, levels = color_levels, cmap = 'jet')
    ax.set_aspect('equal')
    ax.set_xlabel(r'$x$', fontsize=fontsize)
    ax.set_ylabel(r'$y$', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize)
#     axes.title('Time = %.2f' %(index * 10 * advec_var.delta_t_2d))
#     fig.savefig('../../results/2D_Wave_images/%04d' %(index) + '.png')
#     plt.close('all')
#     plt.show()
    return im


In [8]:
results_directory = '../../results/2D_Wave_images'

if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# The directory where h5py files are stored.
h5py_directory = '../../results/2d_hdf5_%02d' %int(params.N_LGL)
path, dirs, files = os.walk(h5py_directory).__next__()
file_count = len(files)
print(h5py_directory)
print(file_count)

shape_u = utils.shape(advec_var.u_e_ij)

for i in np.arange(file_count):
    print(i)

    fig = plt.figure()

    h5py_data = h5py.File('../../results/2d_hdf5_%02d/dump_timestep_%06d' % (int(params.N_LGL), int(i * 20)) + '.hdf5', 'r')
    u_LGL     = af.np_to_af_array(h5py_data['u_i'][:])

    ax1 = plt.subplot2grid((2, 6), (0, 0), colspan = 2, rowspan = 2)
    ax2 = plt.subplot2grid((2, 6), (0, 2), colspan = 2, rowspan = 2)
    ax3 = plt.subplot2grid((2, 6), (0, 4), colspan = 2, rowspan = 2)
    contour_2d(u_LGL[:, :, 0], advec_var, ax1, title = r'$E_z(t = %.2f)$'%(i * 20 * advec_var.delta_t_2d))
    contour_2d(u_LGL[:, :, 1], advec_var, ax2, title = r'$B_x(t = %.2f)$'%(i * 20 * advec_var.delta_t_2d))
    im = contour_2d(u_LGL[:, :, 2], advec_var, ax3, title = r'$B_y(t = %.2f)$'%(i * 20 * advec_var.delta_t_2d))


    plt.tight_layout()
    cb_ax = fig.add_axes([1, 0.3, 0.02, 0.4])
    cbar = fig.colorbar(im, cax=cb_ax)

    fig.savefig('../../results/2D_Wave_images/%04d' %(i) + '.png', bbox_inches='tight', dpi = 400)
    plt.close('all')
    # plt.show()

../../results/2d_hdf5_08
157
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
